In [ ]:
'''
Tips:
If you are trying to learn Selenium/web scraping check these videos out, 
and my straight forward code here should help you figure it out.

install: https://www.youtube.com/watch?v=yhFjXskURFU
demo: https://www.youtube.com/watch?v=oOr1dlC4-Kc
'''

In [357]:
'''
<> On Deck <> Still caught by bot detector, disables point collection after 5 searches
testing with 4 searches
<> pick 50 items from dataframe at random
<> going back to top site and clicking on a news card
<> add in random code to go to page 2-5 results

Thoughts:
Turns out Microsoft Edge doesn't really work on mac. Who knew. Look into remote driver later
consider openning a second tab and duplicating
to use this you must replace login_password and my email

class="cardToggle" when trending news is in search top. shows up at top after third story is chosen.

Idea. instead of waiting use right click paste title
'''
print()

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import pandas as pd
from time import sleep
from random import randint

In [2]:
import airflow

ModuleNotFoundError: No module named 'airflow'

In [2]:
word_bank = './word_bank.csv'
password = '../../Daka/bing_login/pass.csv'

In [3]:
# grabbing reddit text from .csv
df = pd.read_csv(word_bank)

# hiding my password in a local only .csv for github upload
login = pd.read_csv(password)
login_pass = login.passwords[0]

In [4]:
# set user agent
opts = Options()
opts.add_argument("user-agent=The Muffin Man")

# Loading Webpage
driver = webdriver.Chrome(chrome_options=opts)
driver.set_window_size(1280, 720)
driver.get('https://www.bing.com/')


In [5]:
# Logging into account
sleep(2) # sleep 2 to let page load before code runs (avoids skipping)
logger = driver.find_element_by_xpath('//*[@id="id_s"]')
logger.click()

In [6]:
sleep(2)
driver.find_element_by_xpath('//*[@id="i0116"]').clear()
element = driver.find_element_by_xpath('//*//*[@id="i0116"]')
element.send_keys('gnnjume@gmail.com')
element.send_keys(Keys.ENTER)

In [7]:
# checks off remember me box. Selenium memory clears after instance. So not working.
# rememeber = driver.find_element_by_xpath('//*[@id="idChkBx_PWD_KMSI0Pwd"]')
# rememeber.click()

In [8]:
sleep(2)
driver.find_element_by_xpath('//*[@id="i0118"]').clear()
element = driver.find_element_by_xpath('//*[@id="i0118"]')
element.send_keys(login_pass)
element.send_keys(Keys.ENTER)

In [9]:
# Database search loop

# Every loop a random comment is picked as this index counter stacks up.
num=0

for i in range(3): # (df.shape[0]):
    x = randint(1,3)
    y = randint(1,7)
    sleep(randint(3,10))
    if num <= df.shape[0]:   
#         although pressing remember me when loging in is possible, every seleneium instance is new. 
#         However, as this gets more robust I would like reasurance every part is working and that there are no important hickups getting skipped.
        driver.find_element_by_xpath('//*[@id="sb_form_q"]').clear()
        element = driver.find_element_by_xpath('//*[@id="sb_form_q"]')
        element.send_keys(df.posts[num])
        element.send_keys(Keys.ENTER)
        sleep(2)
    
#       reduces the index number to stay within the shape of dataframe
    else:
        num -= randint(15,30)
        
#     Selenium isn't great at mimicing a scroll wheel, but keys work the same. This placeholder scroll will do for now.
    for i in range(x): # 3 is end of page results
        scroll = driver.find_element_by_xpath('/html/body')
        scroll.send_keys(Keys.SPACE)
        sleep(randint(1,3))
        if x+2 <= 4:
            scroll.send_keys(Keys.SHIFT, Keys.SPACE)
            
#     checks out first link, however errors can occur if the link is not the first text. Look into find_element_by_class
    if y <= 3:
        tab_rand = driver.find_element_by_xpath('/html/body')
        tab_rand.send_keys(Keys.TAB, Keys.ENTER)
        sleep(randint(2,5))
        # repeat scroll def here
        driver.back()
   
    # heads back to bing home. I chose the logo over the back button because not being on the landing page will mess up other funcitons
    if y == 5:
        driver.find_element_by_xpath('//*[@id="sb_form"]/a').click()
    
    # addition can be used to make random choice for def start if > df.shape[0] minus random large number
    num += randint(1,30) 

In [10]:
# nav to rewards page
sleep(1)
points_link = '//*[@id="id_rh"]'
click_PL = driver.find_element_by_xpath(points_link)
click_PL.click()
sleep(5)

# not sure why cleaner method wasn't working. Fix later.
click_PL.send_keys(Keys.TAB, Keys.TAB, Keys.TAB, Keys.TAB, Keys.TAB, Keys.ENTER)
sleep(2)


## for some reason these profile xpath is not working
# profile = '//*[@id="header"]/a' (my rewards)
# profile = '//*[@id="cards"]/div[1]/a' (see all)
# click_p = driver.find_element_by_xpath(profile)
# click_p.send_keys(Keys.ENTER) or .click


In [11]:
# click news cards for points
# Xpath version
pt1 = '//*[@id="daily-sets"]/mee-card-group[1]/div/mee-card['
pt2 = ']/div/card-content/mee-rewards-daily-set-item-content/div/div[3]/a'
num = '1'


profile_tab = driver.window_handles[0]
driver.switch_to_window(profile_tab)
for i in range(2):
    sleep(3)

    cardlink = pt1 + num + pt2
#     card_two_link = '//*[@id="daily-sets"]/mee-card-group[1]/div/mee-card[2]/div/card-content/mee-rewards-daily-set-item-content/div/div[3]/a'
    MScard = driver.find_element_by_xpath(cardlink)
    MScard.click()
    sleep(4)
    new_tab = driver.window_handles[1]
    driver.switch_to_window(new_tab)
    driver.close()
    driver.switch_to_window(profile_tab)
#     driver.back()
    num = str(int(num) + 1)

'''                                                        ***
card 1 //*[@id="daily-sets"]/mee-card-group[1]/div/mee-card[1]/div/card-content/mee-rewards-daily-set-item-content/div/div[3]/a
card 2 //*[@id="daily-sets"]/mee-card-group[1]/div/mee-card[2]/div/card-content/mee-rewards-daily-set-item-content/div/div[3]/a
'''   
    
# B4S version

    
    
# random time delay function
# random code. 1 or 0. accidently click username first, sleep(1-3)
# reference rscroll function
# reference back button function
print('')

In [12]:
# Closing up code to prevent multiple instances.
sleep(2)
driver.close()

In [ ]:
# snipit tester

y = 3
driver.find_element_by_xpath('//*[@id="sb_form_q"]').clear()
element = driver.find_element_by_xpath('//*[@id="sb_form_q"]')
element.send_keys(df.posts[30])
element.send_keys(Keys.ENTER)
sleep(2)
if y == 3:
    tab_rand = driver.find_element_by_xpath('/html/body')
    tab_rand.send_keys(Keys.TAB, Keys.ENTER)